<a href="https://colab.research.google.com/github/liadekel/analyzing-big-data/blob/master/Lecture_1_Data_Science_Toolbox_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Structured Data - SQL
### [The Art of Analyzing Big Data - The Data Scientist’s Toolbox - Lecture 1](https://www.ise.bgu.ac.il/labs/fire/lectures.html)
#### By Dr. Michael Fire 

----

The following notebook has three parts. In the first part, we will setup the Kaggle package and use it to download a dataset. In the second part, we will present several examples of how to use Python & SQLite to analyze structure data. Lastly, we see some examples of how to work with Object-Relational Mapping (ORM) and the Pony ORM package.



## 1. Setting Kaggle's Datasets API

This is our first notebook. So first, let's get a Kaggle API key, and create  easy access to download Kaggle's datasets.
More details on Kaggle's API can be found in the following [link](https://github.com/Kaggle/kaggle-api#api-credentials). Moreover, a quick explanation on how to get an API key and use it can be found in the following [link](https://gist.github.com/jayspeidell/d10b84b8d3da52df723beacc5b15cb27)

In [0]:
!mkdir /root/.kaggle/

In [0]:
import json
import os

# Installing the Kaggle package
!pip install kaggle 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
#api_token = {"username":"<Insert Your Kaggle User Name>","key":"<Insert Your Kaggle API key>"}


# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

Let's check if we can use Kaggle API and download the [US Baby Names  dataset](https://www.kaggle.com/kaggle/us-baby-names)

In [0]:
# searching for the dataset
!kaggle datasets list -s baby

ref                                                      title                                         size  lastUpdated          downloadCount  
-------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  
kaggle/us-baby-names                                     US Baby Names                                173MB  2017-11-21 22:18:15          16945  
doyonghoon/baby-data                                     Baby data                                     40KB  2018-04-11 04:35:31            393  
new-york-city/nyc-baby-names                             NYC Baby Names                               136KB  2017-09-08 20:04:20            550  
roopalik/amazon-baby-dataset                             Amazon baby dataset                           18MB  2017-02-01 21:01:16            541  
samrat77/baby-names-dataset                              Baby Names DataSet                            85KB  2019-03-21 06:2

In [0]:
# Creating a dataset directory
!mkdir ./datasets
!mkdir ./datasets/us-baby-name

# download the dataset from Kaggle and unzip it
!kaggle datasets download kaggle/us-baby-names -f database.sqlite -p ./datasets/us-baby-name/database.sqlite
!chdir ./datasets/us-baby-name/database.sqlite/
!unzip ./datasets/us-baby-name/database.sqlite/*.zip  -d ./datasets/us-baby-name/database.sqlite

 94% 123M/131M [00:03<00:00, 21.8MB/s]
100% 131M/131M [00:03<00:00, 40.8MB/s]
/bin/bash: chdir: command not found
Archive:  ./datasets/us-baby-name/database.sqlite/database.sqlite.zip
  inflating: ./datasets/us-baby-name/database.sqlite/database.sqlite  


In [0]:
!unzip ./datasets/us-baby-name/database.sqlite/*.zip  -d ./datasets/us-baby-name/database.sqlite

Archive:  ./datasets/us-baby-name/database.sqlite/database.sqlite.zip
replace ./datasets/us-baby-name/database.sqlite/database.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## 2. Working with SQLite
### Code Example - US Baby Names Dataset

In the first part, we downloaded an SQLite database into Colab. Let's connect to the dataset using the [sqlite3 package](https://docs.python.org/2/library/sqlite3.html): First, let's import the required packages, and answer several questions about the data.

In [0]:
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import os
%matplotlib inline

DB_PATH = './datasets/us-baby-name/database.sqlite'

**Question 1: How many names in the dataset?**

To answer this question, we need to run a query on the database. This can be done by doing following actions:

1.   We connect to the database, which is a local file, using the *connect* function.
2.   We create a *cursor object*.
3.   We call the *execute function* to run a query
4.   We fetch the results using the *fetchone* function


In [0]:
conn = sqlite3.connect(f'{DB_PATH}/database.sqlite') # connecting to the database
c = conn.cursor() # creating a cursor object
print("The number of national names in the dataset: %s" % c.execute("SELECT count(*) FROM NationalNames" ).fetchone()) # execute a query & fetch the results
print("The number of state names in the dataset: %s" % c.execute("SELECT count(*) FROM StateNames" ).fetchone()) 
c.close() # close the cursor

The number of national names in the dataset: 1825433
The number of state names in the dataset: 5647426


**Question 2: How many distinct names in the Washington State? and in New York State?**

We can answer this question, by using the [Select Distinct] statment(https://www.w3schools.com/sql/sql_distinct.asp):

In [0]:
c = conn.cursor()
print("The number of distinct names in WA state is: %s" % c.execute("SELECT count(*) FROM StateNames Where State='WA'" ).fetchone())
print("The number of distinct names in NY state is: %s" % c.execute("SELECT count(*) FROM StateNames Where State='NY'" ).fetchone())
c.close()

The number of distinct names in WA state is: 114240
The number of distinct names in NY state is: 277809


**Question 3: What are the most common/rare names?**

To answer this question, we need to count the number of times each name appeared in the dataset. This can be done by using the following steps: First, we will use the [Group By](https://www.w3schools.com/sql/sql_groupby.asp) statement, and SQL [Sum function](https://www.w3schools.com/sql/sql_count_avg_sum.asp). Second, we will use the [Order By](https://www.w3schools.com/sql/sql_orderby.asp) keyword to order the results. Lastly, we iterate through the retrieved results using a for loop: 

In [0]:
c = conn.cursor()  
rows = c.execute("""SELECT Name, Sum(Count) as TotalNumberOfBabies
             FROM NationalNames
             Group By Name
             Order by Sum(Count)""" ) 
i = 1 
for r in rows:
  if i > 20:
    break
  print(f"{i}. {r[0]}, {r[1]}")
  i += 1
c.close() # close the cursor

1. Aabid, 5
2. Aadhyan, 5
3. Aadian, 5
4. Aadrian, 5
5. Aadrit, 5
6. Aafreen, 5
7. Aagot, 5
8. Aahron, 5
9. Aaiyana, 5
10. Aaja, 5
11. Aakanksha, 5
12. Aakira, 5
13. Aakiyah, 5
14. Aaleeya, 5
15. Aalias, 5
16. Aalicia, 5
17. Aaliyaa, 5
18. Aallyah, 5
19. Aalon, 5
20. Aalyia, 5


Now let's print the names in descending order:

In [0]:
c = conn.cursor()  
rows = c.execute("""SELECT Name, Sum(Count) as TotalNumberOfBabies
             FROM NationalNames
             Group By Name
             Order by Sum(Count) DESC""" ) 
i = 1 
for r in rows:
  if i > 20:
    break
  print(f"{i}. {r[0]}, {r[1]}")
  i += 1
c.close() # close the cursor

1. James, 5129096
2. John, 5106590
3. Robert, 4816785
4. Michael, 4330805
5. Mary, 4130441
6. William, 4071368
7. David, 3590557
8. Joseph, 2580687
9. Richard, 2564867
10. Charles, 2376700
11. Thomas, 2291517
12. Christopher, 2004177
13. Daniel, 1876880
14. Elizabeth, 1606282
15. Patricia, 1575529
16. Matthew, 1558671
17. Jennifer, 1467573
18. George, 1464430
19. Linda, 1454599
20. Barbara, 1437083


We can see that the most common names are James, and the rarest names are names that appeared a total of 5 times. 
*Can you guess why aren't any unique names in the database?*

## 3. Working with Pony ORM

Object-relational mapping (ORM) is very useful for analyzing datasets. I like [Pony ORM](https://ponyorm.org) because it is really easy to use and work with. Moreover, Pony has easy to follow [documentation](https://docs.ponyorm.org). In this part of the notebook, we are going to work with Pony to transfer the database to object and objects into databases. Let's start by installing Pony ORM, and downloading the Pokemon Dataset and convert it into a database.

In [0]:
!pip install pony

     |████████████████████████████████| 286kB 2.8MB/s 
  Created wheel for pony: filename=pony-0.7.12-cp36-none-any.whl size=351612 sha256=cad43c8400456a93c3c801ca3829e9d1198b8021b6c079c9de294cdcb6d0471f
  Stored in directory: /root/.cache/pip/wheels/c8/17/43/4f7d5034e6941b2837f090a788c3abfde02a2c6ddb542e98d7
Successfully built pony


In [0]:
!kaggle datasets list -s pokemon

ref                                                  title                                       size  lastUpdated          downloadCount  
---------------------------------------------------  -----------------------------------------  -----  -------------------  -------------  
abcsds/pokemon                                       Pokemon with stats                          15KB  2016-08-29 06:01:43          44149  
terminus7/pokemon-challenge                          Pokemon- Weedle's Cave                     285KB  2017-09-21 08:43:04          25636  
rounakbanik/pokemon                                  The Complete Pokemon Dataset                47KB  2017-09-29 19:49:32          17226  
vishalsubbiah/pokemon-images-and-types               Pokemon Image Dataset                        2MB  2018-12-17 01:01:39           3795  
thedagger/pokemon-generation-one                     Pokemon Generation One                       2GB  2018-08-02 06:47:33           2393  
kvpratama/pokemon-im

In [0]:
!kaggle datasets download abcsds/pokemon -p ./datasets/
!unzip ./datasets/pokemon.zip -d ./datasets/pokemon/

  0% 0.00/14.9k [00:00<?, ?B/s]
100% 14.9k/14.9k [00:00<00:00, 13.9MB/s]
Archive:  ./datasets/pokemon.zip
  inflating: ./datasets/pokemon/Pokemon.csv  


In [0]:
!ls ./datasets/pokemon

Pokemon.csv


### 3.1 Creating DB using Objects

Now let's use the Pokemon dataset and Pony to create a database. The first step is to create classes that will map to tables in the generate database:

In [0]:
from pony.orm import *
# Creating a new database
db = Database()
db.bind(provider='sqlite', filename='/content/datasets/pokemon/pokemon.pony.db', create_db=True) # notice we can easily use other type of databases instead of SQLite

# Each class will be map to a table
class Trainer(db.Entity):
    name = Required(str)
    age = Required(int)
    pokemons_list = Set('Pokemon') # Connected to the Pokemon class
    
class Pokemon(db.Entity):
    name = Required(str) # cannot be None
    age  = Optional(int)
    attack_power = Required(int)
    defense_power = Required(int)
    trainer = Optional(Trainer) # Connected to the Trainer class

In [0]:
show(Trainer)

class Trainer(Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(str)
    age = Required(int)
    pokemons_list = Set(Pokemon)


Notice that Pony adds a PrimaryKey to the Trainer class. Let's create the actual tables:

In [0]:
set_sql_debug(True) # helps to see what SQL commands are running
db.generate_mapping(create_tables=True) # create tables

GET CONNECTION FROM THE LOCAL POOL
PRAGMA foreign_keys = false
BEGIN IMMEDIATE TRANSACTION
CREATE TABLE "Trainer" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT NOT NULL,
  "age" INTEGER NOT NULL
)

CREATE TABLE "Pokemon" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT NOT NULL,
  "age" INTEGER,
  "attack_power" INTEGER NOT NULL,
  "defense_power" INTEGER NOT NULL,
  "trainer" INTEGER REFERENCES "Trainer" ("id") ON DELETE SET NULL
)

CREATE INDEX "idx_pokemon__trainer" ON "Pokemon" ("trainer")

SELECT "Pokemon"."id", "Pokemon"."name", "Pokemon"."age", "Pokemon"."attack_power", "Pokemon"."defense_power", "Pokemon"."trainer"
FROM "Pokemon" "Pokemon"
WHERE 0 = 1

SELECT "Trainer"."id", "Trainer"."name", "Trainer"."age"
FROM "Trainer" "Trainer"
WHERE 0 = 1

COMMIT
PRAGMA foreign_keys = true
CLOSE CONNECTION


We can see that Pony create the Pokemon & the Trainer tables in the database. Next, we will use the Pandas package to insert data into these tables:

In [0]:
import pandas as pd # we will talk more about pandas in our next lecture
df = pd.read_csv('./datasets/pokemon/Pokemon.csv')
df

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


Let's iterate on the dataframe rows and create a Pokemon object from each row:

In [0]:
import random

for idx, row in df.iterrows():
    Pokemon(name=row['Name'],age=random.randint(0,500),attack_power=row['Sp. Atk'], defense_power=row['Sp. Def'])

We have created 800 objects, one for each row. However, none of them was committed to the DB yet. Let's commit!

In [0]:
commit()

GET NEW CONNECTION
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power") VALUES (?, ?, ?, ?)
['Bulbasaur', 371, 65, 65]

INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power") VALUES (?, ?, ?, ?)
['Ivysaur', 285, 80, 80]

INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power") VALUES (?, ?, ?, ?)
['Venusaur', 4, 100, 100]

INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power") VALUES (?, ?, ?, ?)
['VenusaurMega Venusaur', 412, 122, 120]

INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power") VALUES (?, ?, ?, ?)
['Charmander', 17, 60, 50]

INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power") VALUES (?, ?, ?, ?)
['Charmeleon', 291, 80, 65]

INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power") VALUES (?, ?, ?, ?)
['Charizard', 26, 109, 85]

INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power") VALUES (?, ?, ?, ?)
['CharizardMega C

Now let's add Ash as a trainer object, and add two pokemons trained by Ash:

In [0]:
ash_trainer = Trainer(name='Ash Ketchum', age=10)
pikachu = Pokemon(name='pikachu',age=6,attack_power=50, defense_power=50, trainer=ash_trainer)
charmander =  Pokemon(name='charmander',age=2,attack_power=60, defense_power=50,trainer=ash_trainer)
commit()

BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Trainer" ("name", "age") VALUES (?, ?)
['Ash Ketchum', 10]

INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power", "trainer") VALUES (?, ?, ?, ?, ?)
['pikachu', 6, 50, 50, 1]

INSERT INTO "Pokemon" ("name", "age", "attack_power", "defense_power", "trainer") VALUES (?, ?, ?, ?, ?)
['charmander', 2, 60, 50, 1]

COMMIT


We can now easily retrieve data using objects:

In [0]:
pikachu.trainer.age

10

In [0]:
list(pikachu.trainer.pokemons_list)

[Pokemon[802], Pokemon[801]]

In [0]:
list(pikachu.trainer.pokemons_list)[1].name

'pikachu'

We can also simply update the database using the created objects:

In [0]:
pikachu.age

6

In [0]:
pikachu.age =110
commit()

BEGIN IMMEDIATE TRANSACTION
UPDATE "Pokemon"
SET "age" = ?
WHERE "id" = ?
  AND "name" = ?
  AND "age" = ?
  AND "attack_power" = ?
  AND "defense_power" = ?
  AND "trainer" = ?
[110, 801, 'pikachu', 6, 50, 50, 1]

COMMIT


In [0]:
pikachu.age

110

### 3.2 Creating  Objects from DB:

Using Pony, we can also connect to existing database, and easily search it without even knowing SQL. First let's connect between Pony to existing databse:

In [0]:
from pony.orm import *
db = Database()
db.bind(provider='sqlite', filename='/content/datasets/pokemon/pokemon.pony.db') # notice we don't use create_db=True here

class Trainer(db.Entity):
    name = Required(str)
    age = Required(int)
    pokemons_list = Set('Pokemon')
    
class Pokemon(db.Entity):
    name = Required(str) # cannot be None
    age  = Optional(int)
    attack_power = Required(int)
    defense_power = Required(int)
    trainer = Optional(Trainer)
db.generate_mapping()
set_sql_debug(True)

GET NEW CONNECTION
RELEASE CONNECTION
GET CONNECTION FROM THE LOCAL POOL
SWITCH TO AUTOCOMMIT MODE
SELECT "Pokemon"."id", "Pokemon"."name", "Pokemon"."age", "Pokemon"."attack_power", "Pokemon"."defense_power", "Pokemon"."trainer"
FROM "Pokemon" "Pokemon"
WHERE 0 = 1

SELECT "Trainer"."id", "Trainer"."name", "Trainer"."age"
FROM "Trainer" "Trainer"
WHERE 0 = 1

RELEASE CONNECTION
RELEASE CONNECTION


Using Pony, we can also connect to existing database, and easily search it without even knowing SQL. First, let's connect between Pony to an existing database:

In [0]:
[i.name for i in select(p for p in Pokemon if p.attack_power >100 and p.defense_power > p.attack_power    )]

SELECT "p"."id", "p"."name", "p"."age", "p"."attack_power", "p"."defense_power", "p"."trainer"
FROM "Pokemon" "p"
WHERE "p"."attack_power" > 100
  AND "p"."defense_power" > "p"."attack_power"



['Ho-oh',
 'MetagrossMega Metagross',
 'Latias',
 'LatiasMega Latias',
 'RotomHeat Rotom',
 'RotomWash Rotom',
 'RotomFrost Rotom',
 'RotomFan Rotom',
 'RotomMow Rotom',
 'Florges',
 'Sylveon',
 'Goodra']

We can observe that behind the scenes Pony translate the query into a SQL query and returned a list with the names of the matched pokemons. Let's find the pokemon with the highest attack power:

In [0]:
m = max([i.attack_power for i in select(p for p in Pokemon  )])
m

SELECT "p"."id", "p"."name", "p"."age", "p"."attack_power", "p"."defense_power", "p"."trainer"
FROM "Pokemon" "p"



194

In [0]:
[i.name for i in select(p for p in Pokemon if p.attack_power == m )]

SELECT "p"."id", "p"."name", "p"."age", "p"."attack_power", "p"."defense_power", "p"."trainer"
FROM "Pokemon" "p"
WHERE "p"."attack_power" = ?
[194]



['MewtwoMega Mewtwo Y']

## 4. SQL Big Data
You can use SQL with various RDBMS to analyze tables with millions, tens of millions, or even hundreds of millions of rows. This is sufficient for most usages. However, it is also possible to use "SQL Like" queries to analyze Big Data with billions of rows (or even more). For example, it is possible to use  [HiveQL](https://www.tutorialspoint.com/hive/hiveql_select_where.htm) to analyze extremely large datasets on Hadoop, or [Spark SQL](https://spark.apache.org/sql/) to run SQL like queries on Spark. 

We will show an example of using HiveQL and Spark SQL in a future lecture. For, the last part of this lecture, let's run on a [Kaggle Kernel](https://www.youtube.com/watch?time_continue=1&v=FloMHMOU5Bs&feature=emb_logo), a simple example that utilizes SQL & BigQuery to query large scale datasets:



In [0]:
# see also https://www.kaggle.com/mrisdal/mentions-of-kaggle-on-hacker-news
# Run this on Kaggle Kernel
from google.cloud import bigquery


client = bigquery.Client()

query = """
SELECT title, time_ts
FROM `bigquery-public-data.hacker_news.stories`
WHERE title like '%big%data%'
ORDER BY time DESC
"""

query_job = client.query(query)
iterator = query_job.result(timeout=30)
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
headlines = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 10 headlines
headlines.head(10)

In [0]:
# see also https://www.kaggle.com/dhanushkishore/impact-of-game-of-thrones-on-us-baby-names
from google.cloud import bigquery

client = bigquery.Client()

query = """
SELECT gender, name, sum(number) as totalnames 
FROM `bigquery-public-data.usa_names.usa_1910_current` 
GROUP BY gender, name
Order by totalnames desc
"""

query_job = client.query(query)
iterator = query_job.result(timeout=30)
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
headlines = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 10 headlines
headlines.head(10)

## Further Readings and Practice


*   [SQLZoo](https://sqlzoo.net)
*   [The SQL Murder Mystery](https://mystery.knightlab.com)
*   [W3 School SQLL Tutorial](https://www.w3schools.com/sql/)
*   [PonyORM](https://ponyorm.org)
*   [Kaggle - Intro to SQL](https://www.kaggle.com/learn/intro-to-sql)
*   [Getting Started with SQL and BigQuery notebook](https://www.kaggle.com/dansbecker/getting-started-with-sql-and-bigquery)



  